In [1]:
import awkward as ak
import json
import uproot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import sys
import pickle
import math

from scipy import stats
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution
from scipy import integrate
from scipy.optimize import curve_fit
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint, dual_annealing
from scipy.interpolate import RegularGridInterpolator

import emcee
import iminuit
from iminuit.cost import UnbinnedNLL, UnbinnedNLLNorm, BinnedNLL, NormalConstraint
from iminuit import Minuit

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KDTree
from sklearn.neighbors import KernelDensity

from itertools import permutations
import itertools

from scipy.special import erfi
from scipy.stats import cauchy

import numba as nb

In [2]:
sys.path.append("/Users/jsen/work/Protodune/analysis/pi0_fitter")
from pi0fit.pi0_fit import Pi0Fitter
from pi0fit.shower_model import AnalyticShowerModel, BinnedShowerModel
from pi0fit.pi0_transform import Pi0Transformations
from pi0fit.pi0_fit import Pi0Fitter
from pi0fit.clean_event import CleanEvent
from pi0fit.pi0_model import BinnedPi0Model,Pi0Model
from pi0fit.pi0_likelihood_minimizer import DualAnnealingMinimizer
import pi0fit.fitter_utilities as futil

In [3]:
with open("../config.json", 'r') as f:
        config = json.load(f)

param_fit_file = "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/single_shower_param_fit_v0.pkl"
with open(param_fit_file, 'rb') as f:
    param_fit_dict = pickle.load(f)

In [4]:
config["analytic_shower_model"]["normalization_splines"] = "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/topo_norm_interpolation_v2.pkl"
config["binned_shower_model"]["kde_file"]= "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/charge_pdf_3d_TR_kde_bw0_20_50_1200.pkl"
config["binned_pi0_model"]["charge_hist_file"]= "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/mean_variance_hist_single_gamma_pdf_rotated_cdist_3d_wzero_50_1200.pkl"
config["binned_pi0_model"]["direction_hist_file"]= "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/hist_single_gamma_direction_pdf_rotated_cdist_3d.pkl"
config["pi0_transformations"]["use_fixed_endpoint"] = False
config["pi0_fitter"]["transform_points"] = False
config["pi0_fitter"]["rotate_polar_axis"] = False
config["binned_pi0_model"]["variance_divisor"]= 4.
config["clean_event"]["beam_cos_theta_cut"] = 0.92
config

{'pi0_fitter': {'minimizer': 'DualAnnealingMinimizer',
  'debug': False,
  'truth_comparison': True,
  'use_true_point_transform': False,
  'fit_range': [1670, 1710],
  'fit_all_events': False,
  'return_pi0_only': False,
  'rotate_polar_axis': False,
  'transform_points': False},
 'pi0_minimizers': {'debug': False,
  'fit_pi0_var': False,
  'dual_annealing': {'annealing_temp': 16000,
   'maxiter': 2000,
   'restart_temp_ratio': 0.0002,
   'use_scan_start': False,
   'epi0_limits': [140, 2400],
   'cos_pi0_limits': [0, 1],
   'eg1_limits': [50, 1200],
   'eg2_limits': [50, 1200],
   'theta1_limits': [0, 180],
   'theta2_limits': [0, 180],
   'phi1_limits': [-180, 180],
   'phi2_limits': [-180, 180],
   'c1_limits': [5, 100],
   'c2_limits': [5, 100]}},
 'pi0_transformations': {'debug': False,
  'branches': [],
  'show_shower_plots': False,
  'use_fixed_endpoint': False,
  'shower_direction_bins': 50,
  'true_pi0_start': [-172.5, 300, 115]},
 'binned_pi0_model': {'debug': False,
  'n_ra

In [5]:
pi0_transform = Pi0Transformations(config)
gamma_transform = Pi0Transformations(config)
pi0_model = BinnedPi0Model(config)
pi0_fitter = Pi0Fitter(config)
clean_event = CleanEvent(config)
pi0_minimizer = DualAnnealingMinimizer(config)

Loaded QHist file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/mean_variance_hist_single_gamma_pdf_rotated_cdist_3d_wzero_50_1200.pkl
QHist Bins (20, 20, 20)
Loaded DirHist file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/hist_single_gamma_direction_pdf_rotated_cdist_3d.pkl
DirHist Bins (40, 40, 40)
Random Hist (50, 20, 20, 20)
Loaded QHist file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/mean_variance_hist_single_gamma_pdf_rotated_cdist_3d_wzero_50_1200.pkl
QHist Bins (20, 20, 20)
Loaded DirHist file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/hist_single_gamma_direction_pdf_rotated_cdist_3d.pkl
DirHist Bins (40, 40, 40)
Random Hist (50, 20, 20, 20)
Loaded QHist file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/mean_variance_hist_single_gamma_pdf_rotated_cdist_3d_wzero_50_1200.pkl
QHist Bins (20, 20, 20)
Loaded DirHist file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/hist_single_gamma_direction_pdf_rotated_cdist_3d.pkl
DirHist Bins (40, 40

In [6]:
def two_gamma_with_2conv(evt_record):
    return np.count_nonzero(evt_record["pi0_gamma_endprocess_initial"] == "conv", axis=1) == 2

def two_gamma_with_1conv(evt_record):
    return np.count_nonzero(evt_record["pi0_gamma_endprocess_initial"] == "conv", axis=1) == 1

def add_true_conv_dist(evt_record):
    conv_mask = event_record["pi0_gamma_electron_process_initial"] == "conv" 
    min_dist_x = ak.min(abs(evt_record["pi0_startx_initial"] - evt_record["pi0_gamma_electron_startx_initial"][conv_mask]), axis=2)
    min_dist_y = ak.min(abs(evt_record["pi0_starty_initial"] - evt_record["pi0_gamma_electron_starty_initial"][conv_mask]), axis=2)
    min_dist_z = ak.min(abs(evt_record["pi0_startz_initial"] - evt_record["pi0_gamma_electron_startz_initial"][conv_mask]), axis=2)
    
    #return np.sqrt(min_dist_x**2 + min_dist_y**2 + min_dist_z**2)
    return min_dist_x, min_dist_y, min_dist_z

In [7]:
# file_name_pdspana2 = "/Users/jsen/work/Protodune/analysis/event_data/single_particle_2gev/test_pdspana_beam_pion_n9_2k.root:pduneana/beamana"
# file_name_pdspana = "/Users/jsen/work/Protodune/analysis/event_data/prod4a/prod4a_pdspana_only_pi0_n32.root:pduneana/beamana"

file_name_pdspana2= "/Users/jsen/work/Protodune/analysis/event_data/prod4a/prod4a_pdspana_n500.root:pduneana/beamana"
# file_name_pdspana2 = "/Users/jsen/work/Protodune/analysis/event_data/prod4a/prod4a_pdspana_2_n1000.root:pduneana/beamana"




pdspana_branch = ["event", "run", "subrun", "reco_beam_startX", "reco_beam_startY", "reco_beam_startZ", "reco_beam_endX","reco_beam_endY","reco_beam_endZ", 
                  "reco_beam_calo_endX","reco_beam_calo_endY","reco_beam_calo_endZ",
                  "reco_beam_calo_endDirX", "reco_beam_calo_endDirY", "reco_beam_calo_endDirZ"]

pdspana_branch += ["true_beam_PDG", "true_beam_endProcess", "true_beam_startP", "true_daughter_nPi0", "true_daughter_nPiMinus", "true_daughter_nProton", "true_daughter_nPiPlus", "true_beam_daughter_PDG",
                  "true_beam_endX_SCE", "true_beam_endY_SCE", "true_beam_endZ_SCE", "true_beam_daughter_startP", "true_beam_daughter_startPx", "true_beam_daughter_startPy",
                  "true_beam_daughter_startPz"]

pdspana_branch += ["reco_daughter_allShower_energy", "reco_daughter_allShower_dirX", "reco_daughter_allShower_dirY", "reco_daughter_allShower_dirZ",
                  "reco_daughter_PFP_emScore", "true_beam_startP", "true_beam_endP"]

pdspana_branch += ["reco_daughter_allTrack_startX", "reco_daughter_allTrack_startY", "reco_daughter_allTrack_startZ"]
pdspana_branch += ["reco_daughter_allTrack_endX", "reco_daughter_allTrack_endY", "reco_daughter_allTrack_endZ"]
pdspana_branch += ["reco_daughter_allTrack_Theta", "reco_daughter_allTrack_Phi", "reco_daughter_allTrack_alt_len", "reco_daughter_allTrack_calibrated_dEdX_SCE"]
pdspana_branch += ["reco_daughter_allShower_dirX","reco_daughter_allShower_dirY","reco_daughter_allShower_dirZ", "reco_daughter_PFP_nHits"]
pdspana_branch += ["reco_daughter_allTrack_Chi2_proton", "reco_daughter_allTrack_Chi2_ndof", "reco_daughter_PFP_true_byHits_PDG"]
pdspana_branch += ["reco_daughter_allTrack_Chi2_pion", "reco_daughter_allTrack_Chi2_ndof_pion"]
pdspana_branch += ["true_beam_Pi0_decay_startP"]

pdspana_event_record = uproot.concatenate(files={file_name_pdspana2}, expressions=pdspana_branch)

# pdspana_event_record["valid_true_beam_daughter"] = ((pdspana_event_record["true_beam_daughter_PDG"] < 2000) | (pdspana_event_record["true_beam_daughter_PDG"] == 2212)) & \
#                                                      (pdspana_event_record["true_beam_daughter_PDG"] != 22) & (pdspana_event_record["true_beam_daughter_startP"] > 0.01) & \
#                                                      (pdspana_event_record["true_beam_daughter_PDG"] != 111)
pdspana_event_record["valid_true_beam_daughter"] = ((pdspana_event_record["true_beam_daughter_PDG"] < 2000) | (pdspana_event_record["true_beam_daughter_PDG"] == 2212)) & \
                                                     (pdspana_event_record["true_beam_daughter_startP"] > 0.01) & (pdspana_event_record["true_beam_daughter_PDG"] != 111)

pdspana_event_record["true_cex"] = (pdspana_event_record["true_beam_PDG"] == 211) & (pdspana_event_record["true_beam_endProcess"] == "pi+Inelastic") & \
(pdspana_event_record["true_daughter_nPi0"] == 1) & (pdspana_event_record["true_daughter_nPiPlus"] == 0) & (pdspana_event_record["true_daughter_nPiMinus"] == 0) & (pdspana_event_record["true_daughter_nProton"] > 0)

pdspana_event_record["true_single_pi0"] = (pdspana_event_record["true_beam_PDG"] == 211) & (pdspana_event_record["true_beam_endProcess"] == "pi+Inelastic") & (pdspana_event_record["true_daughter_nPi0"] == 1) 

pdspana_event_record["validation_candidate"] = (pdspana_event_record["true_beam_PDG"] == 211) & (pdspana_event_record["true_beam_endProcess"] == "pi+Inelastic") & \
(pdspana_event_record["true_daughter_nPi0"] == 0) & (pdspana_event_record["true_daughter_nPiPlus"] == 0) & (pdspana_event_record["true_daughter_nPiMinus"] == 0) & (pdspana_event_record["true_daughter_nProton"] > 0)

pdspana_event_record["beam_pion"] = (pdspana_event_record["true_beam_PDG"] == 211) & (pdspana_event_record["true_beam_endProcess"] == "pi+Inelastic") & \
(pdspana_event_record["true_beam_endZ_SCE"] > 0.)     

len(pdspana_event_record)


498

In [8]:
# file_name2 = "/Users/jsen/work/Protodune/analysis/event_data/single_particle_2gev/test_v8_beam_pion_with_pdg_n9_2k.root:trkUtil/points"
# file_name = "/Users/jsen/work/Protodune/analysis/event_data/prod4a/prod4a_only_pi0_n32_all_primary.root:trkUtil/points"
# file_name = "/Users/jsen/work/Protodune/analysis/event_data/prod4a/prod4a_only_pi0_n32_more_cosmic_info.root:trkUtil/points"

file_name2 = "/Users/jsen/work/Protodune/analysis/event_data/prod4a/prod4a_beamana_sp_n500.root:trkUtil/points"
# file_name2 = "/Users/jsen/work/Protodune/analysis/event_data/prod4a/prod4a_beamana_sp_2_n1000.root:trkUtil/points"



# Space Points
branches = ["reco_all_spacePts_X", "reco_all_spacePts_Y", "reco_all_spacePts_Z", "reco_all_spacePts_Integral", "reco_all_spacePts_EMScore"]
#"reco_all_spacePts_MotherPDG"] 

# Pi0 
branches += ["pi0_pdg_initial", "pi0_trackid_initial", "pi0_startpx_initial", "pi0_startpy_initial", "pi0_startpz_initial",
            "pi0_startx_initial", "pi0_starty_initial", "pi0_startz_initial", "pi0_starte_initial", "pi0_gamma_open_angle_initial"]

# Pi0 Decay Gammas
branches += ["pi0_gamma_startx_initial", "pi0_gamma_starty_initial", "pi0_gamma_startz_initial", "pi0_gamma_endx_initial", "pi0_gamma_endy_initial",
             "pi0_gamma_endz_initial", "pi0_gamma_startpx_initial", "pi0_gamma_startpy_initial", "pi0_gamma_startpz_initial", "pi0_gamma_starte_initial",
             "pi0_gamma_pdg_initial", "pi0_gamma_motherid_initial", "pi0_gamma_trackid_initial", "pi0_gamma_process_initial", "pi0_gamma_endprocess_initial"]

# Pi0 Decay Gamma Pair e-e+ or Compton electron
branches += ["pi0_gamma_electron_startx_initial", "pi0_gamma_electron_starty_initial", "pi0_gamma_electron_startz_initial",
             "pi0_gamma_electron_startpx_initial", "pi0_gamma_electron_startpy_initial", "pi0_gamma_electron_startpz_initial",
             "pi0_gamma_electron_starte_initial", "pi0_gamma_electron_pdg_initial", "pi0_gamma_electron_motherid_initial",
             "pi0_gamma_electron_trackid_initial", "pi0_gamma_electron_process_initial"]


branches += ["pi0_gamma_electron_startx_initial_sce", "pi0_gamma_electron_starty_initial_sce", "pi0_gamma_electron_startz_initial_sce"]
# branches += ["n_pi0", "total_event_count", "truth_beam_pdg"]

has_cosmics = True

## Cosmic Stuff
if has_cosmics:
    branches += ["total_event_count", "cosmic_pfp_spacePts_X", "cosmic_pfp_spacePts_Y", "cosmic_pfp_spacePts_Z", "cosmic_pfp_spacePts_ID"]
    
    branches += ["cosmic_pfp_start_X", "cosmic_pfp_start_Y", "cosmic_pfp_start_Z", "cosmic_pfp_end_X", "cosmic_pfp_end_Y", "cosmic_pfp_end_Z",
                 "cosmic_pfp_IsPrimary", "cosmic_pfp_IsClearCosmic", "cosmic_pfp_IsBeam", "cosmic_pfp_ID", "cosmic_pfp_nSpPts"]


raw_event_record = uproot.concatenate(files={file_name2}, expressions=branches)

# Only consider decays with 2 gammas
#two_gamma_decay_mask = np.count_nonzero(raw_event_record["pi0_gamma_process_initial"] == "Decay", axis=1) == 2
event_record = raw_event_record#[two_gamma_decay_mask]

tmp_zeros = ak.zeros_like(event_record["reco_all_spacePts_X"])
tmp_ones = ak.ones_like(event_record["reco_all_spacePts_X"])
event_record["reco_daughter_PFP_shower_spacePts_R"] = tmp_zeros
event_record["reco_daughter_PFP_shower_spacePts_Theta"] = tmp_zeros
event_record["reco_daughter_PFP_shower_spacePts_Phi"] = tmp_zeros

"""
Add additional columns
"""
event_record["two_gamma_with_2conv"] = two_gamma_with_2conv(evt_record=event_record)
event_record["two_gamma_with_1conv"] = two_gamma_with_1conv(evt_record=event_record)
event_record["true_decay_gamma_conv_startx"], event_record["true_decay_gamma_conv_starty"],event_record["true_decay_gamma_conv_startz"] = add_true_conv_dist(evt_record=event_record)

print("nEvts:", len(event_record))
print("PDSPAna nEvt:", len(pdspana_event_record))

# pdspana_event_record = pdspana_event_record[event_record["total_event_count"]-1]
print("PDSPAna nEvt:", len(pdspana_event_record))

nEvts: 498
PDSPAna nEvt: 498
PDSPAna nEvt: 498


In [9]:
num = 498#498#9000
cex_events = [i for i in range(num) if pdspana_event_record["true_cex",i]]
print(len(cex_events))
print(cex_events)

pi0_events = [i for i in range(num) if pdspana_event_record["true_single_pi0",i]]
print(len(pi0_events))
print(pi0_events)

24
[7, 31, 57, 74, 138, 145, 154, 177, 194, 204, 214, 234, 253, 263, 307, 343, 364, 365, 371, 431, 464, 477, 479, 486]
81
[7, 14, 15, 16, 17, 20, 31, 42, 56, 57, 74, 77, 86, 97, 116, 117, 136, 138, 143, 144, 145, 151, 154, 165, 170, 171, 177, 180, 184, 188, 194, 197, 204, 207, 208, 212, 214, 215, 216, 221, 231, 234, 253, 256, 263, 273, 282, 286, 290, 295, 296, 297, 307, 308, 328, 339, 340, 343, 345, 346, 349, 359, 364, 365, 371, 384, 400, 406, 417, 418, 431, 433, 441, 443, 458, 464, 470, 476, 477, 479, 486]


In [10]:
def get_pi0_points(event_record, pdspana_event_record, event, pts_shift, return_spherical, cosmics=False, get_gammas=True):

    pts = np.vstack((ak.to_numpy(event_record["reco_all_spacePts_X",event]) - pts_shift[0],
             ak.to_numpy(event_record["reco_all_spacePts_Y",event]) - pts_shift[1],
             ak.to_numpy(event_record["reco_all_spacePts_Z",event]) - pts_shift[2])).T
    if get_gammas:
        sstart = 0.
    if cosmics:
        cosmic_pts = np.vstack((ak.to_numpy(event_record["cosmic_pfp_spacePts_X", event]) - pts_shift[0],
                                ak.to_numpy(event_record["cosmic_pfp_spacePts_Y", event]) - pts_shift[1],
                                ak.to_numpy(event_record["cosmic_pfp_spacePts_Z", event]) - pts_shift[2])).T

    if get_gammas:
        ssdir = np.vstack((ak.to_numpy(event_record["pi0_gamma_startpx_initial", event]),
                        ak.to_numpy(event_record["pi0_gamma_startpy_initial", event]),
                        ak.to_numpy(event_record["pi0_gamma_startpz_initial", event]))).T
        
        sdir1 = ssdir[0] / (np.sqrt(ssdir[0] @ ssdir[0]))
        sdir2 = ssdir[1] / (np.sqrt(ssdir[1] @ ssdir[1]))

        spherical_sdir1 = futil.single_to_spherical(v=sdir1, rotate_polar_axis=False).T
        spherical_sdir2 = futil.single_to_spherical(v=sdir2, rotate_polar_axis=False).T

    em_mask = 0#ak.to_numpy(event_record["reco_all_spacePts_EMScore", event]) > em_thresh

        
    spherical_pts_tmp = futil.single_to_spherical(v=pts.T).T
    spherical_pts = np.vstack((spherical_pts_tmp[:,0], np.degrees(spherical_pts_tmp[:,1]), np.degrees(spherical_pts_tmp[:,2]))).T


    if cosmics:
        if return_spherical:
            spherical_cosmic_pts_tmp = futil.single_to_spherical(v=cosmic_pts.T).T
            spherical_cosmic_pts = np.vstack((spherical_cosmic_pts_tmp[:,0], np.degrees(spherical_cosmic_pts_tmp[:,1]), np.degrees(spherical_cosmic_pts_tmp[:,2]))).T
            if get_gammas:
                return spherical_pts, spherical_sdir1, spherical_sdir2, sstart, spherical_cosmic_pts
            else:
                return spherical_pts, spherical_cosmic_pts
        else:
            if get_gammas:
                return pts, sdir1, sdir2, sstart, cosmic_pts
            else:
                return pts, cosmic_pts
    else:
        if return_spherical:
            if get_gammas:
                return spherical_pts, em_mask, spherical_sdir1, spherical_sdir2, sstart
            else:
                return spherical_pts
        else:
            if get_gammas:
                return pts, em_mask, sdir1, sdir2, sstart
            else:
                return pts, em_mask

def calculate_shower_energy(epi0, open_angle):
    mgg2 = 135. * 135.
    num = np.sqrt(2. * mgg2 * (np.cos(open_angle) - 1.) + epi0 * epi0 * (1. - np.cos(open_angle)) ** 2) \
          + epi0 * (np.cos(open_angle) - 1.)
    denom = 2. * (np.cos(open_angle) - 1.)
    calculated_e1 = num / denom

    if np.isnan(calculated_e1):
        print("Nan, splitting energy")
        return epi0 / 2., epi0 / 2.

    return calculated_e1, epi0 - calculated_e1

def get_vertex(event, use_true_endpoint):

    xt,yt,zt = pdspana_event_record["true_beam_endX_SCE",event], pdspana_event_record["true_beam_endY_SCE",event], pdspana_event_record["true_beam_endZ_SCE",event]    
    xr,yr,zr = pdspana_event_record["reco_beam_endX",event], pdspana_event_record["reco_beam_endY",event], pdspana_event_record["reco_beam_endZ",event]
    dr = np.sqrt((xt-xr)**2 + (yt-yr)**2 + (zt-zr)**2)
    print("Δx/y/z/r", np.round(xt-xr,3), "/", np.round(yt-yr,3), "/", np.round(zt-zr,3), "/", dr)

    vertex = [xt, yt, zt] if use_true_endpoint else [xr, yr, zr]
    return vertex, dr


In [ ]:
use_true_vtx = False

fit_res_list3 = []
fit_vals_list3 = []

for evt in cex_events:
    
    print("Evt:", evt)
    evt_charge = ak.to_numpy(event_record["reco_all_spacePts_Integral",evt])
    pi0_fitter.lower_range,pi0_fitter.upper_range = evt, evt + 1
    xyz_vertex, dr = get_vertex(event=evt, use_true_endpoint=use_true_vtx)

    if dr > 40 or np.any(pdspana_event_record["true_beam_Pi0_decay_startP",evt]*1.e3 < 45) or len(pdspana_event_record["true_beam_Pi0_decay_startP",evt]) > 2: 
        print("Vertex BAD!  Δr:", dr)
        print("OR Too low E", pdspana_event_record["true_beam_Pi0_decay_startP",evt]*1.e3)
        continue
    
    cartesian_pts,cosmic_pts = get_pi0_points(event_record=event_record, pdspana_event_record=pdspana_event_record, event=evt, pts_shift=xyz_vertex, return_spherical=False, cosmics=True, get_gammas=False)
    spherical_pts = get_pi0_points(event_record=event_record, pdspana_event_record=pdspana_event_record, event=evt, pts_shift=xyz_vertex, return_spherical=True, cosmics=False, get_gammas=False)

    valid_cosmic_mask = clean_event.cosmic_selection(event_record=event_record, evt=evt, hit_cut=200)
    cleaned_spherical_pts, cleaned_charge = clean_event.clean_event(spherical_pts=spherical_pts, cartesian_pts=cartesian_pts, charge_pts=evt_charge, cosmic_pts=cosmic_pts[valid_cosmic_mask], xyz_vertex=xyz_vertex)
    no_proton_mask = clean_event.proton_cut(event_record=pdspana_event_record, spherical_pts=cleaned_spherical_pts, event=evt, xyz_shift=xyz_vertex)
    
    pi0_pts = np.vstack((cleaned_spherical_pts[no_proton_mask][:,0], cleaned_spherical_pts[no_proton_mask][:,1], cleaned_spherical_pts[no_proton_mask][:,2], 
                         cleaned_charge[no_proton_mask], np.ones_like(cleaned_charge[no_proton_mask]))).T
    
    fit_vals,fit_res = pi0_fitter.fit_pi0(all_event_record=event_record, pi0_points=pi0_pts)
    if fit_vals is None or fit_res is None:
        continue
    fit_res_list3.append(list(fit_res.values()))
    fit_vals_list3.append(list(fit_vals.values()))
    

Evt: 7
Δx/y/z/r -6.279 / 12.204 / 18.61 / 23.12286411098742
Cosmic Pre 45581
Cosmic Post (remaining): 7232
nShower-like Particles: 3 Selected: dict_keys([0, 1, 2])
0 )  theta/phi/nhit/Pchi2/dr 60 / 130 / 844 / 179.55 / 27.53  [ 22 ]
1 )  theta/phi/nhit/Pchi2/dr 164 / 50 / 103 / 236.44 / 22.29  [ 211 ]
2 )  theta/phi/nhit/Pchi2/dr 32 / -62 / 82 / 2.04 / 23.96  [ 2212 ]
3 )  theta/phi/nhit/Pchi2/dr -57238 / -57238 / 12 / 1.0 / 2063.47  [ 2212 ]
Removed Protons: []
######## Event: 7
True Eπ0: 803.4102647907869
True Gamma Eγ: [268, 535]
True OA: 20.519152593768638
True cos_pi0: 0.9715546175765383
True Gamma Energy 1/2 268.33 / 535.08
True Gamma Theta 1/2 71.32 / 55.91
True Gamma Phi 1/2 117.33 / 132.55
C1/C2: 0.0 / 0.0
MADE IT!!
Bounds: [(50, 1200), (50, 1200), (50, 1200), (0, 180), (0, 180), (0, 180), (-180, 180), (-180, 180), (-180, 180)]
Start Point: [500 800 200  30  60 110  50 -20 100]
Event Charge Fill
Pre-Qhist 43123.41138380766


/Users/jsen/opt/anaconda3/envs/pi0_fitting/lib/python3.10/site-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'bins' of function 'BinnedPi0Model.set_bin_charge_3d'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../pi0fit/pi0_model.py", line 388:
    @nb.njit(error_model="numpy")
    def set_bin_charge_3d(hist, X, Y, Z, Q, bins):
    ^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/Users/jsen/opt/anaconda3/envs/pi0_fitting/lib/python3.10/site-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'bins' of function 'BinnedPi0Model.distance_cut_2d'.

For more information visit https://numba.readthedocs.io/en/stable/refere

Qhist 27225.992479264736 Calo Energy 1090.514959006616
Charge Bins (20, 20, 20)  Dir Bins (40, 40, 40)
Starting 2Shower Minimization!


/Users/jsen/opt/anaconda3/envs/pi0_fitting/lib/python3.10/site-packages/scipy/optimize/_differentialevolution.py:382: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,
/Users/jsen/opt/anaconda3/envs/pi0_fitting/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Iter: 0 Mutation_upper: 0.55 Fmin1: 20.38958477072345
Iter: 1 Mutation_upper: 0.55 Fmin1: 20.168524731333047
Iter: 2 Mutation_upper: 0.55 Fmin1: 21.984617605293987
Selected Min: 20.168524731333047
+--------+---------+--------+--------+-------+-------+--------+--------+------------+-----+-----+---------+
|  Eπ0   | cosθ_π0 |  Eγ1   |  Eγ2   |  θγ1  |  θγ2  |   ϕ1   |   ϕ2   | Open Angle |  C1 |  C2 | istruth |
+--------+---------+--------+--------+-------+-------+--------+--------+------------+-----+-----+---------+
| 803.41 |   0.49  | 268.33 | 535.08 | 71.32 | 55.91 | 117.33 | 132.55 |   20.52    | 0.0 | 0.0 |   1.0   |
| 894.62 |   0.64  | 365.82 | 528.8  | 55.14 | 47.11 | 105.97 | 126.26 |   17.66    | 5.0 | 5.0 |   0.0   |
+--------+---------+--------+--------+-------+-------+--------+--------+------------+-----+-----+---------+
+------------------+--------------------+------------------+------------------+------------+------------+
| Eπ0: reco/true-1 | cosθ_π0: true-reco | Eγ1: re

/Users/jsen/opt/anaconda3/envs/pi0_fitting/lib/python3.10/site-packages/scipy/optimize/_differentialevolution.py:382: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,
/Users/jsen/opt/anaconda3/envs/pi0_fitting/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Iter: 0 Mutation_upper: 0.55 Fmin1: 22.7852286657332
Iter: 1 Mutation_upper: 0.55 Fmin1: 22.785229080684662
Iter: 2 Mutation_upper: 0.55 Fmin1: 22.78610303366577
Selected Min: 22.7852286657332
+--------+---------+--------+-------+-------+--------+-------+--------+------------+-----+-----+---------+
|  Eπ0   | cosθ_π0 |  Eγ1   |  Eγ2  |  θγ1  |  θγ2   |   ϕ1  |   ϕ2   | Open Angle |  C1 |  C2 | istruth |
+--------+---------+--------+-------+-------+--------+-------+--------+------------+-----+-----+---------+
| 171.46 |   0.41  | 121.93 | 49.52 | 46.76 | 143.52 | 17.14 | -67.25 |   120.57   | 0.0 | 0.0 |   1.0   |
| 373.29 |   0.88  | 348.54 | 24.75 | 32.01 |  64.0  |  73.0 | -81.0  |   93.23    | 5.0 | 5.0 |   0.0   |
+--------+---------+--------+-------+-------+--------+-------+--------+------------+-----+-----+---------+
+------------------+--------------------+------------------+------------------+------------+------------+
| Eπ0: reco/true-1 | cosθ_π0: true-reco | Eγ1: reco/true-1 

/Users/jsen/opt/anaconda3/envs/pi0_fitting/lib/python3.10/site-packages/scipy/optimize/_differentialevolution.py:382: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


Iter: 0 Mutation_upper: 0.55 Fmin1: 19.343573271619366


In [ ]:
ones = np.ones(len(pi0_pts), dtype=bool)
shower1 = np.vstack((ones, ones * np.radians(34), ones * np.radians(120))).T
tmp_pts = np.vstack((ones, np.radians(pi0_pts[:,1]), np.radians(pi0_pts[:,2]))).T
tmp = shower_point_cos1 = futil.spherical_dot(shower1, tmp_pts)
tmp_mask = tmp > 0.9
print(tmp.shape)
print(tmp_mask.shape)

pi0_pts[:,3][tmp_mask].shape

In [43]:
# test(epi0=565.48, open_angle=np.radians(27.75))
print(len(pi0_pts))
print()
energy_from_shower_direction(pts=pi0_pts, open_angle=np.radians(17.83), theta1=56.94, theta2=47.96, phi1=104.91, phi2=124.43, print_stuff=True)

1119

Selected Count: 290 / 298
S1: E 381 MeV
S2: E 381 MeV


/Users/jsen/work/Protodune/analysis/pi0_fitter/pi0fit/pi0_likelihood_minimizer.py:292: RuntimeWarning: invalid value encountered in sqrt
  num = np.sqrt(2. * mgg2 * (np.cos(open_angle) - 1.) + epi0 * epi0 * (1. - np.cos(open_angle)) ** 2) \


(763.512873655055, 381.7564368275275, 381.7564368275275)

In [42]:
def energy_from_shower_direction(pts, open_angle, theta1, theta2, phi1, phi2, print_stuff=True):

    ones = np.ones(len(pts), dtype=bool)

    tmp_pts = np.vstack((ones, np.radians(pts[:, 1]), np.radians(pts[:, 2]))).T
    shower1 = np.vstack((ones, ones * np.radians(theta1), ones * np.radians(phi1))).T
    shower2 = np.vstack((ones, ones * np.radians(theta2), ones * np.radians(phi2))).T

    shower_point_cos1 = futil.spherical_dot(shower1, tmp_pts)
    shower_point_cos2 = futil.spherical_dot(shower2, tmp_pts)

    charge_save_mask1 = shower_point_cos1 > 0.92
    charge_save_mask2 = shower_point_cos2 > 0.92

    if print_stuff: print("Selected Count:", np.count_nonzero(charge_save_mask1), "/",
                          np.count_nonzero(charge_save_mask2))

    qsum1 = np.sum(pts[:, 3][charge_save_mask1])
    qsum2 = np.sum(pts[:, 3][charge_save_mask2])
    qtotal = np.sum(pts[:, 3][charge_save_mask1 & charge_save_mask2])

    calo_epi0 = pi0_model.calo_to_energy(charge=qtotal)

    tmp_e1, tmp_e2 = pi0_minimizer.calculate_shower_energy(epi0=calo_epi0, open_angle=open_angle)
    calc_energy1, calc_energy2 = (tmp_e1, tmp_e2) if qsum1 > qsum2 else (tmp_e2, tmp_e1)

    if print_stuff:
        print("S1: E", int(calc_energy1), "MeV")
        print("S2: E", int(calc_energy2), "MeV")

    return calc_energy1 + calc_energy2, calc_energy1, calc_energy2